In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# ==============================================================================
# 单元格 1：环境设置与API Key配置
# ==============================================================================
print("正在安装/升级Google AI Generative Language库...")
!pip install -q --upgrade google-generativeai

print("\n正在配置API Key...")
import google.generativeai as genai
from google.colab import userdata

try:
    # 从Colab密文中安全地获取API Key
    api_key = userdata.get('GEMINI_API_KEY')
    if not api_key:
        raise userdata.SecretNotFoundError("API Key is empty or not found in secrets.")

    # 使用 configure 方法配置 API Key
    genai.configure(api_key=api_key)

    print("API Key配置成功！")

except userdata.SecretNotFoundError:
    print("错误：未在Colab密文中找到名为 'GEMINI_API_KEY' 的密文，或密文为空。请先添加您的API Key。")
except Exception as e:
    print(f"配置时发生错误: {e}")

正在安装/升级Google AI Generative Language库...

正在配置API Key...
API Key配置成功！


In [2]:
# 获取可用模型列表
models = genai.list_models()
for model in models:
    print(f"模型名称: {model.name}, 支持的任务: {model.supported_generation_methods}")

模型名称: models/embedding-gecko-001, 支持的任务: ['embedText', 'countTextTokens']
模型名称: models/gemini-2.5-flash, 支持的任务: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
模型名称: models/gemini-2.5-pro, 支持的任务: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
模型名称: models/gemini-2.0-flash-exp, 支持的任务: ['generateContent', 'countTokens', 'bidiGenerateContent']
模型名称: models/gemini-2.0-flash, 支持的任务: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
模型名称: models/gemini-2.0-flash-001, 支持的任务: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
模型名称: models/gemini-2.0-flash-exp-image-generation, 支持的任务: ['generateContent', 'countTokens', 'bidiGenerateContent']
模型名称: models/gemini-2.0-flash-lite-001, 支持的任务: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
模型名称: models/gemini-2.0-flash-lite, 支持的任务: ['generateContent', 'countTokens', 'createCachedContent

In [ ]:
# ==============================================================================
# 单元格 2：项目配置 (Configuration)
# ==============================================================================
import os
import re
import json
import time
from tqdm.notebook import tqdm

class Config:
    # --- 本地路径配置 (请根据您的Google Drive路径修改) ---
    GDRIVE_PROJECT_PATH = "/content/drive/MyDrive/data"
    VIDEO_INPUT_DIR = os.path.join(GDRIVE_PROJECT_PATH, "Crash-1500")
    OUTPUT_DIR = os.path.join(GDRIVE_PROJECT_PATH, "VQA_outputs")
    JSON_OUTPUT_FILE = os.path.join(OUTPUT_DIR, "generated_vqa.json")

    # --- 模型配置 ---
    # Gemini 1.5 Pro的最新模型标识符
    MODEL_ID = "models/gemini-2.5-pro"

    # --- Prompt模板 (保持不变) ---
    SCENE_ANALYSIS_PROMPT = """你是一位顶级的交通事故分析专家。你的任务是基于我提供的视频，对事故场景进行全面的分析。

请严格按照以下的Markdown格式，总结你的分析结果：

### 事故场景综合分析
**1. 环境要素:**
   - **光照与天气:**
   - **道路类型:**
**2. 关键参与者与事发前动态:**
   - **参与者A:**
   - **参与者B:**
**3. 碰撞过程:**
   - **关键行为:**
   - **碰撞描述:**
**4. 事故核心原因推断:**
   - **主要原因:**
""" # 为节省空间省略，请使用之前的Prompt
    VQA_GENERATION_PROMPT = """你是一位顶级的交通安全领域Benchmark出题专家。
你的任务是基于我提供的“视频”和“事故场景综合分析”，为该事故创建一套包含6个问题的视觉问答（VQA）数据。

**输出要求:**
请严格以一个JSON代码块的格式输出，不要有任何其他多余的话。JSON结构必须是包含6个问题对象的列表。语言必须为英文。

**出题详细指令:**
- **维度**: 必须严格按照以下6个维度各出一个问题：`Weather & Light`, `Traffic Environment`, `Road Configuration`, `Accident Type`, `Accident Cause`, `Accident Prevention`.
- **答案设计**: 每个问题必须有4个选项，必须有唯一的正确答案。另外三个选项必须是“高质量的干扰项”——即看似合理但与视频事实不符的描述。
---
**输入信息:**

**1. 事故场景综合分析:**
{scene_analysis_text}

**2. 原始视频:**
[视频内容已作为输入提供]
---
请现在开始生成符合上述所有要求的JSON数据。
""" # 为节省空间省略，请使用之前的Prompt

# 实例化配置
cfg = Config()

# 挂载Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 确保输出目录存在
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
print("配置加载完成，输出目录已确认。")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
配置加载完成，输出目录已确认。


In [ ]:
# ==============================================================================
# 单元格 3：核心功能函数 (Core Helper Functions)
# ==============================================================================

def upload_video_file(local_video_path: str):
    """
    上传本地视频文件到Google AI API，并等待其处理完成。
    返回一个可用于Prompt的文件对象。
    """
    print(f"  正在上传视频: {os.path.basename(local_video_path)}...")
    video_file = genai.upload_file(path=local_video_path)

    # 等待视频处理完成
    while video_file.state.name == "PROCESSING":
        print("  视频正在处理中，请稍候...")
        time.sleep(10)
        video_file = genai.get_file(video_file.name)

    if video_file.state.name == "FAILED":
        raise ValueError("视频文件处理失败。")

    print(f"  视频 '{video_file.display_name}' 上传并处理成功！")
    return video_file

def run_gemini_pipeline_for_video(video_file, model) -> dict:
    """为单个已上传的视频文件执行完整的两阶段生成流程。"""

    # 阶段一：场景分析
    print("    -> 阶段1: 正在进行场景分析...")
    analysis_response = model.generate_content([cfg.SCENE_ANALYSIS_PROMPT, video_file])
    analysis_text = analysis_response.text

    # 阶段二：VQA生成
    print("    -> 阶段2: 正在生成VQA...")
    vqa_prompt = cfg.VQA_GENERATION_PROMPT.format(scene_analysis_text=analysis_text)
    vqa_response = model.generate_content([vqa_prompt, video_file])

    # 从模型输出中解析JSON
    match = re.search(r"```json\s*\n(.*?)\n\s*```", vqa_response.text, re.DOTALL)
    if match:
        try:
            vqa_json = json.loads(match.group(1))
            return {
                "generated_analysis": analysis_text,
                "generated_vqa": vqa_json
            }
        except json.JSONDecodeError as e:
            print(f"    警告: JSON解析失败 - {e}")
            return None
    else:
        print("    警告: 在模型输出中未找到格式化的JSON块。")
        return None

print("核心功能函数定义完成。")

核心功能函数定义完成。


In [ ]:
# ==============================================================================
# 单元格 4：执行主流程 (Main Execution Loop)
# ==============================================================================

# 初始化模型
model = genai.GenerativeModel(cfg.MODEL_ID)

# 获取所有视频文件列表 (用于存在性检查)
all_video_files_in_dir = set([f for f in os.listdir(cfg.VIDEO_INPUT_DIR) if f.lower().endswith(('.mp4', '.avi', '.mov'))])

# 读取需要处理的视频列表从CSV文件
filtered_videos_path = os.path.join(cfg.GDRIVE_PROJECT_PATH, "filtered_videos.csv")
video_files_to_process_ordered = [] # 存储从CSV读取并过滤后，保持CSV顺序的视频列表
if os.path.exists(filtered_videos_path):
    try:
        import pandas as pd
        df_filtered = pd.read_csv(filtered_videos_path, header=None) # 假设第一列是视频文件名，没有header
        # 格式化第一列的视频名称为6位数字，并确保读取的视频文件名在实际的视频目录中存在
        video_files_from_csv = [f"{int(x):06d}.mp4" for x in df_filtered.iloc[:, 0].tolist()]
        # 仅保留在实际视频目录中存在的视频，并保持CSV中的顺序
        video_files_to_process_ordered = [f for f in video_files_from_csv if f in all_video_files_in_dir]

        print(f"从 {filtered_videos_path} 读取到 {len(video_files_from_csv)} 个视频文件名。")
        print(f"其中 {len(video_files_to_process_ordered)} 个在视频目录中存在，将按照CSV顺序处理。")
        print("--- 待处理的视频列表（来自CSV筛选后，按CSV顺序）---")
        print(video_files_to_process_ordered)
        print("----------------------------------------")
    except Exception as e:
        print(f"读取或解析 {filtered_videos_path} 失败: {e}")
        # 如果读取失败，回退到处理目录中所有视频的逻辑 (此时顺序可能不稳定)
        video_files_to_process_ordered = sorted(list(all_video_files_in_dir)) # 尝试按名称排序作为回退
        print(f"回退：将处理视频目录中的所有视频，共 {len(video_files_to_process_ordered)} 个。")


else:
    print(f"未找到过滤视频列表文件: {filtered_videos_path}，将处理视频目录中的所有视频。")
    # 如果文件不存在，处理目录中所有视频 (此时顺序可能不稳定)
    video_files_to_process_ordered = sorted(list(all_video_files_in_dir)) # 尝试按名称排序作为回退
    print(f"将处理视频目录中的所有视频，共 {len(video_files_to_process_ordered)} 个。")


all_results = []
if os.path.exists(cfg.JSON_OUTPUT_FILE):
    with open(cfg.JSON_OUTPUT_FILE, 'r') as f:
        all_results = json.load(f)
    processed_videos = {item['video_name'] for item in all_results}
    # 仅处理那些在 filtered_videoes.csv 中且尚未处理的视频，保持原有顺序
    video_files_to_process = [f for f in video_files_to_process_ordered if f not in processed_videos]
    print(f"发现已处理 {len(processed_videos)} 个视频，将继续处理剩余的 {len(video_files_to_process)} 个。")
    print("--- 待处理的视频列表（剔除已处理后，按CSV顺序）---")
    print(video_files_to_process)
    print("----------------------------------------")
else:
     # 如果输出文件不存在，则处理从CSV读取的完整列表
     video_files_to_process = video_files_to_process_ordered


uploaded_files = [] # 用于存储本次运行上传的所有文件，以便最后清理

if not video_files_to_process:
    print("没有需要处理的新视频。")
else:
    # 处理过滤后的视频列表
    print(f"将处理 {len(video_files_to_process)} 个视频。")

    for video_name in tqdm(video_files_to_process, desc="处理视频"):
        local_video_path = os.path.join(cfg.VIDEO_INPUT_DIR, video_name)
        print(f"\n--- 正在处理: {video_name} ---")

        try:
            # 1. 上传视频并等待处理
            video_file_handle = upload_video_file(local_video_path)
            uploaded_files.append(video_file_handle) # 记录下来以便清理

            # 2. 运行Gemini流水线
            result = run_gemini_pipeline_for_video(video_file_handle, model)

            if result:
                result['video_name'] = video_name
                all_results.append(result)
                print(f"  成功处理 {video_name} 并获得结果。")
            else:
                print(f"  处理 {video_name} 失败，未获得有效结果。")

        except Exception as e:
            print(f"  处理视频 {video_name} 时发生严重错误: {e}")

        # 3. 定期保存进度
        # 保存逻辑已根据用户需求调整，每处理完一个就保存（为了应对Colab断连），但最终保存会覆盖
        print(f"\n--- 正在保存当前进度 ({len(all_results)} 个视频)... ---")
        # 为了保持原有视频的处理顺序，这里在保存时，需要先将新结果添加到all_results，
        # 然后根据最初的video_files_to_process_ordered列表对all_results进行排序后再保存。
        # 注意：这里假设all_results中的每个字典都有一个'video_name'键
        try:
            # 创建一个字典，key是video_name，value是对应的结果字典
            results_dict = {item['video_name']: item for item in all_results}
            # 按照最初的期望处理顺序，重构all_results列表
            # 确保只包含在 original_video_files_to_process_ordered 中的结果
            # 并且只包含在 results_dict 中已有的结果
            sorted_results = [results_dict[video_name] for video_name in video_files_to_process_ordered if video_name in results_dict]

            with open(cfg.JSON_OUTPUT_FILE, 'w', encoding='utf-8') as f:
                json.dump(sorted_results, f, indent=4, ensure_ascii=False)
        except Exception as e:
             print(f"  保存进度时发生排序或写入错误: {e}. 将直接保存未经排序的当前结果.")
             with open(cfg.JSON_OUTPUT_FILE, 'w', encoding='utf-8') as f:
                json.dump(all_results, f, indent=4, ensure_ascii=False)


    # 4. 最终保存 (虽然上面循环里也在保存，这里再做一次确保)
    print("\n--- 视频处理完毕，正在进行最终保存... ---")
    # 最终保存时也进行排序
    try:
        results_dict = {item['video_name']: item for item in all_results}
        sorted_results = [results_dict[video_name] for video_name in video_files_to_process_ordered if video_name in results_dict]
        with open(cfg.JSON_OUTPUT_FILE, 'w', encoding='utf-8') as f:
            json.dump(sorted_results, f, indent=4, ensure_ascii=False)
    except Exception as e:
        print(f"  最终保存时发生排序或写入错误: {e}. 将直接保存未经排序的最终结果.")
        with open(cfg.JSON_OUTPUT_FILE, 'w', encoding='utf-8') as f:
            json.dump(all_results, f, indent=4, ensure_ascii=False)


    # 5. 清理本次上传的文件
    print("\n--- 正在清理本次上传到Google AI服务器的临时文件... ---")
    for f in uploaded_files:
        try:
            genai.delete_file(f.name)
            print(f"  已删除文件: {f.display_name}")
        except Exception as e:
            print(f"  删除文件 {f.display_name} 失败: {e}")

    print(f"\n流程全部完成！结果已保存在: {cfg.JSON_OUTPUT_FILE}")

从 /content/drive/MyDrive/data/filtered_videos.csv 读取到 344 个视频文件名。
其中 344 个在视频目录中存在，将按照CSV顺序处理。
--- 待处理的视频列表（来自CSV筛选后，按CSV顺序）---
['000003.mp4', '000018.mp4', '000023.mp4', '000031.mp4', '000033.mp4', '000039.mp4', '000048.mp4', '000059.mp4', '000066.mp4', '000074.mp4', '000078.mp4', '000083.mp4', '000087.mp4', '000109.mp4', '000115.mp4', '000130.mp4', '000136.mp4', '000137.mp4', '000142.mp4', '000156.mp4', '000157.mp4', '000159.mp4', '000161.mp4', '000163.mp4', '000166.mp4', '000167.mp4', '000177.mp4', '000179.mp4', '000181.mp4', '000182.mp4', '000185.mp4', '000187.mp4', '000188.mp4', '000190.mp4', '000191.mp4', '000193.mp4', '000198.mp4', '000199.mp4', '000202.mp4', '000206.mp4', '000214.mp4', '000215.mp4', '000220.mp4', '000234.mp4', '000243.mp4', '000244.mp4', '000246.mp4', '000247.mp4', '000250.mp4', '000251.mp4', '000255.mp4', '000256.mp4', '000259.mp4', '000260.mp4', '000264.mp4', '000267.mp4', '000274.mp4', '000276.mp4', '000278.mp4', '000286.mp4', '000300.mp4', '000306.mp4', '00

处理视频:   0%|          | 0/102 [00:00<?, ?it/s]


--- 正在处理: 001063.mp4 ---
  正在上传视频: 001063.mp4...
  视频正在处理中，请稍候...
  视频 '001063.mp4' 上传并处理成功！
    -> 阶段1: 正在进行场景分析...
    -> 阶段2: 正在生成VQA...
  成功处理 001063.mp4 并获得结果。

--- 正在保存当前进度 (243 个视频)... ---

--- 正在处理: 001064.mp4 ---
  正在上传视频: 001064.mp4...
  视频正在处理中，请稍候...
  视频 '001064.mp4' 上传并处理成功！
    -> 阶段1: 正在进行场景分析...
    -> 阶段2: 正在生成VQA...
  成功处理 001064.mp4 并获得结果。

--- 正在保存当前进度 (244 个视频)... ---

--- 正在处理: 001070.mp4 ---
  正在上传视频: 001070.mp4...
  视频正在处理中，请稍候...
  视频 '001070.mp4' 上传并处理成功！
    -> 阶段1: 正在进行场景分析...
    -> 阶段2: 正在生成VQA...
  成功处理 001070.mp4 并获得结果。

--- 正在保存当前进度 (245 个视频)... ---

--- 正在处理: 001073.mp4 ---
  正在上传视频: 001073.mp4...
  视频正在处理中，请稍候...
  视频 '001073.mp4' 上传并处理成功！
    -> 阶段1: 正在进行场景分析...
    -> 阶段2: 正在生成VQA...
  成功处理 001073.mp4 并获得结果。

--- 正在保存当前进度 (246 个视频)... ---

--- 正在处理: 001074.mp4 ---
  正在上传视频: 001074.mp4...
  视频正在处理中，请稍候...
  视频 '001074.mp4' 上传并处理成功！
    -> 阶段1: 正在进行场景分析...
    -> 阶段2: 正在生成VQA...
  成功处理 001074.mp4 并获得结果。

--- 正在保存当前进度 (247 个视频)... ---

--- 正在处理: 0010